In [1]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

In [2]:
#data = pd.read_csv('/home/demange/code/thomasbordier/forecast_energy_consumption/raw_data/eco2mix-regional-cons-def.csv',delimiter=';')
data = pd.read_csv('../raw_data/eco2mix-regional-cons-def.csv',delimiter=';')

/home/demange/.pyenv/versions/lewagon/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (27,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data_PACA = data[data['Région']=="Provence-Alpes-Côte d'Azur"]

In [4]:
data_PACA[data_PACA['Consommation (MW)'].isna()] #reperage des Nan

,Code INSEE région,Région,Nature,Date,Heure,Date - Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),...,TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),Column 30
8,93,Provence-Alpes-Côte d'Azur,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962435,93,Provence-Alpes-Côte d'Azur,Données consolidées,2022-05-06,00:00,2022-05-06T00:00:00+02:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-,ND,-,ND,NaN


In [5]:
imputer = SimpleImputer(strategy="mean") # Instantiate a SimpleImputer object with your strategy of choice
imputer.fit(data_PACA[['Consommation (MW)']]) # Call the "fit" method on the object
data_PACA['Consommation (MW)'] = imputer.transform(data_PACA[['Consommation (MW)']]) # Call the "transform" method on the object

/tmp/ipykernel_10843/2840541328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_PACA['Consommation (MW)'] = imputer.transform(data_PACA[['Consommation (MW)']]) # Call the "transform" method on the object


In [6]:
data_PACA = data_PACA.groupby(by=['Date']).sum()

In [7]:
data_PACA

,Code INSEE région,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Pompage (MW),Bioénergies (MW),Ech. physiques (MW),...,Eolien offshore,TCO Thermique (%),TCH Thermique (%),TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),Column 30
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,4464,257058.278726,15175.0,0.0,372.0,405.0,61254.0,-4.0,1808.0,173268.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0
2013-01-02,4464,280064.000000,18369.0,0.0,1404.0,2655.0,70681.0,-12.0,1717.0,185143.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0
2013-01-03,4464,285526.000000,24755.0,0.0,626.0,2984.0,72282.0,0.0,2357.0,182412.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0
2013-01-04,4464,286799.000000,17085.0,0.0,1028.0,2570.0,68995.0,0.0,2988.0,194018.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0
2013-01-05,4464,267848.000000,15479.0,0.0,1557.0,2898.0,62143.0,0.0,3051.0,182618.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-11,4464,218296.000000,34225.0,0.0,1909.0,10139.0,40944.0,-306.0,3115.0,128276.0,...,0.0,750.65,1257.37,0.0,0.0,41.85,1968.08,215.63,613.37,0.0
2022-05-12,4464,209770.000000,47313.0,0.0,1695.0,15998.0,40585.0,-306.0,3177.0,101324.0,...,0.0,1075.23,1738.16,0.0,0.0,38.92,1747.49,359.01,967.83,0.0
2022-05-13,4464,201061.000000,74913.0,0.0,1215.0,18002.0,41576.0,0.0,3561.0,61809.0,...,0.0,1795.44,2752.11,0.0,0.0,29.28,1252.63,419.87,1089.04,0.0


In [8]:
data_PACA.to_csv('../raw_data/data_energy.csv')